<a href="https://colab.research.google.com/github/RavindrapalRathore/Remote-Sensing-Camp-with-python/blob/main/remote_sensing_with_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install geopandas

In [2]:
# the regulars
import pandas as pd
import geopandas as gpd

# earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

In [5]:
#Trigger the authentication flow
ee.Authenticate()

# Initialize the libary
ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ccVNeTikb9tZG8zFgb-npeWadlrj903SQ863yIHx5Fs&tc=XSEPmRSfq3fA1THyZKLuQILe7LbgkVVPDtb06sqi0D8&cc=kTTaFmhE-k7tsfj_SSu6H-NasyXv4f4HwCUBDAaoAFk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhUYkJtMBhvTwEc0e7ZWd0R9Rxb-jBhxilSWf1j3FXVxo9PEQY-HDvg

Successfully saved authorization token.


## Define filters

In [6]:
# coordinates of the camp fire
lat = 39.444012
lon = -121.833619

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2018-10-01'

# end date
end_date = '2019-01-31'


## Get Landsat 8 data

In [7]:
# get the satellite data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
.filterBounds(poi)\
.filterDate(start_date,end_date)

In [8]:
# how many images did we get?
print('Totalnumber:', landsat.size().getInfo())

Totalnumber: 8


In [9]:
# information about the first image in our collection
landsat.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7661, 7791],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 500385, 0, -30, 4423215]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7661, 7791],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 500385, 0, -30, 4423215]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7661, 7791],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 500385, 0, -30, 4423215]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7661, 7791],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 500385, 0, -30, 4423215]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
 

In [10]:
# what about cloud cover of our first image?
landsat.first().get('CLOUD_COVER').getInfo()


0.05

In [11]:
# when was this image taken?
landsat.first().get('DATE_ACQUIRED').getInfo()

'2018-10-07'

## Bands


In [12]:
# what bands did we get?
landsat.first().bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [13]:
# put the images in a list
landsat_list = landsat.toList(landsat.size());

## Display satellite image


In [14]:
# set some parameters for the images
parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 800, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

In [15]:
# create an empty data container
data = []

# loop through each image and display it
for i in range(landsat.size().getInfo()):

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()

    # print the image info
    print('Image #',i,date,'Cloud cover:',cloud)

    # display the image
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

    # data to list
    this_data = [i,date,cloud]

    # append the data
    data.append(this_data)


# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 0 2018-10-07 Cloud cover: 0.05


Image # 1 2018-10-23 Cloud cover: 73.04


Image # 2 2018-11-08 Cloud cover: 11.83


Image # 3 2018-11-24 Cloud cover: 67.16


Image # 4 2018-12-10 Cloud cover: 56.09


Image # 5 2018-12-26 Cloud cover: 5.99


Image # 6 2019-01-11 Cloud cover: 80.06


Image # 7 2019-01-27 Cloud cover: 5.21


In [16]:
# our data in a dataframe
df

,Image #,Date,Cloud Cover
0,0,2018-10-07,0.05
1,1,2018-10-23,73.04
2,2,2018-11-08,11.83
3,3,2018-11-24,67.16
4,4,2018-12-10,56.09
5,5,2018-12-26,5.99
6,6,2019-01-11,80.06
7,7,2019-01-27,5.21


## Selecting images, zooming in
Now that we have inspected our collection of images, we can pick and choose which ones are relevant for our study. Ideally, we want to have images for before and after the fire to be able to assess the level of damage.

We also want to create an ROI (region of interest) and zoom in to the area of interest. We do so by appying a 20km buffer around our POI.

In [17]:
# create a list of images we want (before, during, after)
landsat_sequence = [0,2,5]

In [18]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(20000) # meters

In [19]:
parameters = {
                'min': 6000,
                'max': 16000,
                'dimensions': 800,
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
                'region':roi
             }

In [20]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()

    print('Image #',i,date,'Cloud cover:',cloud)

    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 0 2018-10-07 Cloud cover: 0.05


Image # 2 2018-11-08 Cloud cover: 11.83


Image # 5 2018-12-26 Cloud cover: 5.99


<div class="alert alert-info">
Take a moment to inspect the three images above. What do they tell you? What do they NOT tell you?
</div>

## Normalized Difference Vegetation Index (NDVI)

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, often from a space platform, assessing whether or not the target being observed contains live green vegetation.

### Calculating NDVI in Google Earth Engine

In [21]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [22]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # print some information
    print('Image #',i,date)

    # display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))

Image # 0 2018-10-07


Image # 2 2018-11-08


Image # 5 2018-12-26


## Folium

In [23]:
# a simple folium map
import folium

m = folium.Map(location=[lat,lon])
m

In [24]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [25]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']),
                        ndvi_parameters,
                        name=date)

# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

## Save the folium map as an html file

In [26]:
my_map.save('camp.html')